<a href="https://colab.research.google.com/github/chetools/CHE4061_Fall2024/blob/main/NRTL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.4 MB/s eta 0:00:00


In [2]:
from importnb import Notebook
with Notebook():
    from che5 import Props

import numpy as np
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)
from scipy.optimize import root_scalar
from scipy.optimize import root
from scipy.special import expit, logit

In [3]:
p=Props(['Acetone','Ethanol', 'Isopropanol', 'Water'])

In [4]:
def gamma(x,T):
    tau = p.NRTL_A + p.NRTL_B/T + p.NRTL_C*np.log(T) + p.NRTL_D*T
    G=np.exp(-p.NRTL_alpha*tau)
    xG = x@G
    xtauG_xG = (x@(tau*G))/xG
    return np.exp(xtauG_xG + x@((G*(tau - xtauG_xG[None,:])/xG[None,:]).T))

In [5]:
def dewP_ideal(y, T):
    P=1./(np.sum(y/p.Pvap(T)))
    return P, y*P/p.Pvap(T)

def dewT_ideal(y, P):

    def P_dev(T):
        return dewP_ideal(y, T)[0] - P
    T = root(P_dev, 300.).x[0]

    return T,  y*P/p.Pvap(T)

In [6]:
def bubbleP_NRTL(x, T):
    Pi= x*gamma(x,T)*p.Pvap(T)
    P=np.sum(Pi)
    return P, Pi/P

def bubbleT_NRTL(x, P):

    def f(T):
        return bubbleP_NRTL(x,T)[0]-P

    #mole-fraction weighted boiling points of each component at P
    #boiling points determined via Clausius Clapeyron, using the Hvap at the normal bp
    #for each component.  p.Hvap returns the heat of vaporization of all components for each
    #temperature if an array of temperatures is given.
    Tguess=np.dot(x,1/(1/p.Tbn-np.log(P/101325)*R/np.diagonal(p.Hvap(p.Tbn))))

    T=root_scalar(f, x0=Tguess, method='secant').root
    Pi= x*gamma(x,T)*p.Pvap(T)
    P=np.sum(Pi)

    return T, Pi/P



In [7]:
z = np.array([0.1,0.2,0.3,0.4])
P = 3e5
R=8.314
bubbleT_NRTL(z,P)


(Array(377.80220917, dtype=float64),
 Array([0.25464361, 0.1853556 , 0.28067204, 0.27932875], dtype=float64))

In [8]:
def dewP_NRTL(y, T):
    def f(vec):
        P = vec[0]
        x = expit(vec[1:])  #Ensures that mole fractions are between 0 and 1

        fug_eqs = x*gamma(x,T) * p.Pvap(T)  - y*P
        xsum_eq = 1. - np.sum(x)

        return np.r_[fug_eqs, xsum_eq]

    #Assume ideal liquid dewP calculation for initial guess of P and liquid phase composition
    Pguess, xguess= dewP_ideal(z,T)

    #f (function to zero) maps values from -inf to inf, to values between 0 and 1
    #so xguess is mapped via logit which is the inverse function of expit
    v0 = np.r_[Pguess, logit(xguess)]
    res=root(f, v0)
    return res.x[0], expit(res.x[1:])

In [9]:
def dewT_NRTL(y, P):
    def f(vec):
        T = vec[0]
        x = expit(vec[1:])  #Ensures that mole fractions are between 0 and 1

        fug_eqs = x*gamma(x,T) * p.Pvap(T)  - y*P
        xsum_eq = 1. - np.sum(x)

        return np.r_[fug_eqs, xsum_eq]

    #Assume ideal liquid dewP calculation for initial guess of P and liquid phase composition
    Tguess, xguess= dewT_ideal(z,P)

    #f (function to zero) maps values from -inf to inf, to values between 0 and 1
    #so xguess is mapped via logit which is the inverse function of expit
    v0 = np.r_[Tguess, logit(xguess)]
    res=root(f, v0)
    return res.x[0], expit(res.x[1:])

In [10]:
dewP_NRTL(z, 373)

(215679.4631248687, array([0.02121585, 0.13299478, 0.17452966, 0.67125971]))

In [11]:
dewT_NRTL(z, 215679)

(372.99993756950886, array([0.02121583, 0.13299476, 0.17452962, 0.67125978]))

In [12]:
def flash_idealPT(z, P, T):

    K=p.Pvap(T)/P
    def rachford(VF):
        return np.sum(z*(K-1)/(VF*(K-1) +1))

    res=root_scalar(rachford, bracket=(0,1))
    VF = res.root
    x=z/(1-VF + K *VF)
    y=K*x
    return x, y, VF

In [13]:
def flash_NRTL_PT(z, P, T, maxiter = 100, tol=1e-12):

    dewP, dewx = dewP_NRTL(z, T)
    bubbleP, bubbley = bubbleP_NRTL(z,T)

    xguess = (P-dewP)/(bubbleP-dewP) * (z - dewx) +  dewx

    for i in range(maxiter):
        K=gamma(xguess,T)*p.Pvap(T)/P

        def rachford(VF):
            return np.sum(z*(K-1)/(VF*(K-1) +1))

        res=root_scalar(rachford, bracket=(0,1))
        VF = res.root
        x=z/(1-VF + K *VF)
        if (np.linalg.norm(xguess-x)<tol):
            break
        xguess = x

    y=K*x
    return x, y, VF, i

In [ ]:
def flash_NRTL_PQ(z, P, Q=0.):




In [48]:
z=np.array([0.1, 0.2, 0.3, 0.4])
FT = 420.
FP = bubbleP_NRTL(z, FT)[0]  #Feed pressure is at bubble pressure
FH = p.Hl(z, FT)

flashP=FP/2

In [52]:
bubbleP_NRTL(z,FT), dewP_NRTL(z, FT)

((Array(966265.83465294, dtype=float64),
  Array([0.19549943, 0.19793845, 0.30147042, 0.3050917 ], dtype=float64)),
 (880585.2939825576, array([0.03561801, 0.15095412, 0.20497964, 0.60844823])))

In [54]:
FP = (bubbleP_NRTL(z,FT)[0]+dewP_NRTL(z, FT)[0])/2
FP

Array(923425.56431775, dtype=float64)

In [55]:
flash_NRTL_PT(z, FP, FT)


(Array([0.05890853, 0.19105812, 0.28089262, 0.46914073], dtype=float64),
 Array([0.13246011, 0.20706362, 0.31509383, 0.34538245], dtype=float64),
 0.5586755692839904,
 30)

Rachford Rice:
For a flash calculation at a given T and P, we want to solve for the compositions of each phase as well as the fraction of the feed that is vapor (or liquid).  To perform this flash calculation the two key relationships are the overall mass balance F = V + L, zF = xL + yV, y=Kx.  Combining these equations and noting that the sum of mole fractions add to 1, results in the Rachford-Rice expression (with a bit algebraic manipulation).  Sum y - sum x = 0

If we have an ideal system, then Ki=Pvap,i(T) / P.  In a non-ideal system, the K=gamma(x,T) * Pvap,i / P.

A dew P calculation yields a Pressure, and also the composition of the first liquid drop.  A bubble P calculation also yields another Pressure value, and the liquid phase composition is the feed composition.  For a flash to yield 2 phases, our pressure has to lie between the dewP and bubbleP.  So we linearly interpolate between the 2 liquid compositions based on where the flash P is relative to the bubbleP and dewP.  

With that interpolated liquid phase composition as a guess, we proceed with a standard rachford rice calculation for the vapor fraction.